In [1]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.93 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.8/112.6 GB disk)


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("mdpimagedataset").project("mdp_image_grp49")
version = project.version(6)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.8
    Uninstalling idna-3.8:
      Successfully uninstalled idna-3.8
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.93, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to mdp_image_grp49-6 in yolov8:: 100%|██████████| 19054/19054 [00:04<00:00, 3879.28it/s]


In [3]:
dataset.location

'/content/mdp_image_grp49-6'

In [4]:
# overwriting file paths to yaml file
data_yaml_content = """
train: /content/mdp_image_grp49-6/train/images
val: /content/mdp_image_grp49-6/valid/images
test: /content/mdp_image_grp49-6/test/images

nc: 31
names: ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'bullseye', 'circle', 'down', 'left', 'right', 'up']

roboflow:
  workspace: mdpimagedataset
  project: mdp_image_grp49
  version: 3
  license: CC BY 4.0
  url: https://universe.roboflow.com/mdpimagedataset/mdp_image_grp49/dataset/6
"""

# Write to the file
with open('/content/mdp_image_grp49-6/data.yaml', 'w') as file:
    file.write(data_yaml_content)

In [5]:
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data='/content/mdp_image_grp49-6/data.yaml' \
epochs=25 \
imgsz=640\
device=0\

100% 21.5M/21.5M [00:00<00:00, 37.5MB/s]
Ultralytics YOLOv8.2.93 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0\ (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/mdp_image_grp49-6/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0\, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

In [6]:
from google.colab import files
files.download('/content/runs/detect/train/weights/last.pt')
files.download('/content/runs/detect/train/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# uploaded = files.upload()
# !ls
# import shutil

# # Move the uploaded checkpoint file to the appropriate directory
# shutil.move('last.pt', '/content/runs/detect/train/weights/last.pt')
# shutil.move('best.pt', '/content/runs/detect/train/weights/best.pt')

In [7]:
# validate w new model
!yolo task=detect \
mode=val \
model='/content/runs/detect/train/weights/best.pt' \
data='/content/mdp_image_grp49-6/data.yaml' \
device=0

Ultralytics YOLOv8.2.93 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,137,581 parameters, 0 gradients, 28.5 GFLOPs
val: Scanning /content/mdp_image_grp49-6/valid/labels.cache... 376 images, 0 backgrounds, 0 corrupt: 100% 376/376 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 24/24 [00:07<00:00,  3.34it/s]
                   all        376        449      0.957      0.935       0.97      0.822
                     1          9          9      0.943      0.778      0.884      0.694
                     2         14         14      0.986      0.929      0.952      0.732
                     3         26         26      0.957          1      0.995       0.82
                     4         10         10      0.978          1      0.995      0.964
                     5         18         18          1      0.956      0.995      0.799
                     6         19      

In [8]:
# predict test data with model
!yolo task=detect \
mode=predict \
model='/content/runs/detect/train/weights/best.pt' \
conf=0.5 \
source='/content/mdp_image_grp49-6/test/images'\
device=0

Ultralytics YOLOv8.2.93 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,137,581 parameters, 0 gradients, 28.5 GFLOPs

image 1/360 /content/mdp_image_grp49-6/test/images/IMG-20240829-WA0009_jpg.rf.dc43419d3eb2b237ab572764d443fef6.jpg: 640x640 1 2, 1 3, 16.3ms
image 2/360 /content/mdp_image_grp49-6/test/images/IMG-20240829-WA0014_jpg.rf.76af91226c492a696f456118ece1f9a6.jpg: 640x640 1 6, 16.3ms
image 3/360 /content/mdp_image_grp49-6/test/images/IMG-20240829-WA0018_jpg.rf.79bbdea82f31146b07841c1813989e98.jpg: 640x640 1 6, 1 9, 16.3ms
image 4/360 /content/mdp_image_grp49-6/test/images/IMG-20240829-WA0025_jpg.rf.9703dcec657f9b1ebebef1baf7c7989b.jpg: 640x640 1 5, 16.3ms
image 5/360 /content/mdp_image_grp49-6/test/images/IMG-20240829-WA0030_jpg.rf.e88c154c545c75a2fe4da61ccb24cbf7.jpg: 640x640 (no detections), 16.3ms
image 6/360 /content/mdp_image_grp49-6/test/images/IMG-20240829-WA0034_jpg.rf.1ac6c02f97edc9cfeb167fa84e91548b.jpg: 640x640 1 8

In [12]:
# export / upload weights
project.version(6).deploy(model_type="yolov8", model_path=f'/content/runs/detect/train/')

Dependency ultralytics==8.0.196 is required but found version=8.2.93, to fix: `pip install ultralytics==8.0.196`
Would you like to continue with the wrong version of ultralytics? y/n: y
View the status of your deployment at: https://app.roboflow.com/mdpimagedataset/mdp_image_grp49/6
Share your model with the world at: https://universe.roboflow.com/mdpimagedataset/mdp_image_grp49/model/6
